# Credenciales

In [2]:
from google.cloud import bigquery
import numpy as np
from numpy.random import seed
from numpy.random import randn
from scipy.stats import shapiro
##%load_ext google.cloud.bigquery
%env GOOGLE_APPLICATION_CREDENTIALS = claveserviapp.json
client = bigquery.Client()

env: GOOGLE_APPLICATION_CREDENTIALS=claveserviapp.json


## Query para buscar los SKU con menos transacciones

In [3]:
query = """
SELECT
  EAN,
  COUNT(EAN) AS count_transaction,
  EXTRACT(WEEK
  FROM
    FechaTransaccion) AS Week
FROM
  `servi-app.BI.JULIO2019_NORMALIDAD_MALIANZA`
GROUP BY
  EAN,
  Week
ORDER BY
  count_transaction DESC
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = query_job.to_dataframe()

In [81]:
df_new = df[(df["count_transaction"] > 5) & (df["count_transaction"] < 500)]

In [127]:
df_new[df_new["count_transaction"] == 8]

,EAN,count_transaction,Week
27682,7702040000202,8,26
27683,7702129019309,8,26
27684,7709990375541,8,26
27685,7707324390093,8,26
27686,7702129076531,8,26
27687,7702047038314,8,26
27688,7709641031857,8,26
27689,7707356370476,8,26
27690,7709823580388,8,26
27691,7702129103480,8,26


## Descriptive are somewhat discouraging with 500 it will be hard to achieve normality 

In [8]:
df_new.sort_values(by=['count_transaction']).describe()

,count_transaction,Week
count,32503.000000,32503.000000
mean,53.746054,27.956004
std,80.618119,1.372917
min,6.000000,26.000000
25%,10.000000,27.000000
50%,21.000000,28.000000
75%,56.000000,29.000000
max,499.000000,30.000000


## The real question is how many values go missing given the rules and if normality is assured. Let answer the normality test first.


### TOP 10 values with fewer transactions

In [12]:
list(zip(list(df_new.sort_values(by=['count_transaction']).head(10)['Week']), list(df_new.sort_values(by=['count_transaction']).head(10)['EAN'])))

[(28, '7702173301009'),
 (28, '7707196101162'),
 (28, '7709990998405'),
 (28, '7707283880543'),
 (28, '7704200547077'),
 (28, '7703518040263'),
 (28, '7702032052332'),
 (28, '7707186854740'),
 (28, '7702011004505'),
 (29, '7702367000015')]

## For one product in a low selling week

In [4]:
query = """
SELECT
  *,
  EXTRACT(WEEK
  FROM
    FechaTransaccion) AS Week
FROM
  `servi-app.BI.JULIO2019_NORMALIDAD_MALIANZA`
WHERE
  EAN IN ( '7707186370493',
    '7702061400609',
    '7709527390726',
    '7752910505016',
    '7709106661957',
    '7707200710359',
    '7707334530038',
    '7702310048095',
    '7702011081186',
    '7509546054032',
    '7709093609345',
    '7702057075101',
    '7707279031713',
    '7702088165048',
    '7702152101606',
    '7702007056303',
    '7702025125173',
    '7702581050247',
    '7702168114409',
    '7709977217277',
    '7702354930974'
    )
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = query_job.to_dataframe()

In [126]:
ventas_d = np.array(list(atransaction['totalventa']))
precioventa =np.array(list(atransaction['precioventa']))
cantidad = np.array(list(atransaction['cantidad']))
np.std(ventas_d)*3 + np.median(ventas_d)

11043.436266998306

## QUESO CIMITARRA TAJADO 

In [61]:
atransaction = df[(df["Week"] == 26) & (df["EAN"]=="7709093609345")]

In [62]:
atransaction

,Codigo,Sale_id,cantidad,precioventa,totalventa,Fechatransaccion,EAN,CiudadM,Tipologia_comercial,DESCRIPCION_FINAL,PRESENTACION,FABRICANTE,MARCA,DPTO,SECCION,CATEGORIA,SUBCATEGORIA,Week
0,262887,1562020774671,2,7000,7000,2019-07-01 17:39:00+00:00,7709093609345,BUCARAMANGA,MINIMERCADO,QUESO CIMITARRA TAJADO MOZZARELLA X 10 UND,10 UND,OTROS FABRICANTES,CIMITARRA,PGC,PLS,LACTEOS/DERIVADOS/HUEVOS,QUESO,26
40,262758,1562024232673,1,3800,3800,2019-07-01 18:37:00+00:00,7709093609345,BUCARAMANGA,MINIMERCADO,QUESO CIMITARRA TAJADO MOZZARELLA X 10 UND,10 UND,OTROS FABRICANTES,CIMITARRA,PGC,PLS,LACTEOS/DERIVADOS/HUEVOS,QUESO,26
75,262887,1562118686161,1,3500,3500,2019-07-02 20:51:00+00:00,7709093609345,BUCARAMANGA,MINIMERCADO,QUESO CIMITARRA TAJADO MOZZARELLA X 10 UND,10 UND,OTROS FABRICANTES,CIMITARRA,PGC,PLS,LACTEOS/DERIVADOS/HUEVOS,QUESO,26
105,262758,1562290069345,1,3800,3800,2019-07-04 20:27:00+00:00,7709093609345,BUCARAMANGA,MINIMERCADO,QUESO CIMITARRA TAJADO MOZZARELLA X 10 UND,10 UND,OTROS FABRICANTES,CIMITARRA,PGC,PLS,LACTEOS/DERIVADOS/HUEVOS,QUESO,26
147,262758,1562028978024,1,3800,3800,2019-07-01 19:56:00+00:00,7709093609345,BUCARAMANGA,MINIMERCADO,QUESO CIMITARRA TAJADO MOZZARELLA X 10 UND,10 UND,OTROS FABRICANTES,CIMITARRA,PGC,PLS,LACTEOS/DERIVADOS/HUEVOS,QUESO,26
148,262758,1562115790295,1,3800,3800,2019-07-02 20:03:00+00:00,7709093609345,BUCARAMANGA,MINIMERCADO,QUESO CIMITARRA TAJADO MOZZARELLA X 10 UND,10 UND,OTROS FABRICANTES,CIMITARRA,PGC,PLS,LACTEOS/DERIVADOS/HUEVOS,QUESO,26
178,262887,1562367978007,1,3500,3500,2019-07-05 18:06:00+00:00,7709093609345,BUCARAMANGA,MINIMERCADO,QUESO CIMITARRA TAJADO MOZZARELLA X 10 UND,10 UND,OTROS FABRICANTES,CIMITARRA,PGC,PLS,LACTEOS/DERIVADOS/HUEVOS,QUESO,26
219,262758,1562182800878,1,3800,3800,2019-07-03 14:40:00+00:00,7709093609345,BUCARAMANGA,MINIMERCADO,QUESO CIMITARRA TAJADO MOZZARELLA X 10 UND,10 UND,OTROS FABRICANTES,CIMITARRA,PGC,PLS,LACTEOS/DERIVADOS/HUEVOS,QUESO,26
274,261962,1562418736847,1,3800,3800,2019-07-06 08:12:00+00:00,7709093609345,BUCARAMANGA,TIENDA GRANDE,QUESO CIMITARRA TAJADO MOZZARELLA X 10 UND,10 UND,OTROS FABRICANTES,CIMITARRA,PGC,PLS,LACTEOS/DERIVADOS/HUEVOS,QUESO,26
277,262758,1562209479041,1,3800,3800,2019-07-03 22:04:00+00:00,7709093609345,BUCARAMANGA,MINIMERCADO,QUESO CIMITARRA TAJADO MOZZARELLA X 10 UND,10 UND,OTROS FABRICANTES,CIMITARRA,PGC,PLS,LACTEOS/DERIVADOS/HUEVOS,QUESO,26


In [63]:
ventas_d = np.array(list(atransaction['totalventa']))
precioventa =np.array(list(atransaction['precioventa']))
cantidad = np.array(list(atransaction['cantidad']))

### Shapiro Tests

In [9]:
# Shapiro-Wilk Test
def shapiro_t(data):
    stat, p = shapiro(data)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')



### NONE IS GAUSSIAN!!!!

In [65]:
for datt in [ventas_d, precioventa, cantidad]:
    shapiro_t(datt)

Statistics=0.400, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.400, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.284, p=0.000
Sample does not look Gaussian (reject H0)


### Given the rule you should be eliminating 1/15 of the sample (with code: 262887); far from ideal. Moreover, the value delete is nota mistake

In [66]:
np.std(ventas_d)*3 + np.median(ventas_d)

6289.979919597747

# For other products

### PAPAS ANDINA

In [72]:
atransaction = df[(df["Week"] == 26) & (df["EAN"]=="7707279031713")]

In [73]:
atransaction

,Codigo,Sale_id,cantidad,precioventa,totalventa,Fechatransaccion,EAN,CiudadM,Tipologia_comercial,DESCRIPCION_FINAL,PRESENTACION,FABRICANTE,MARCA,DPTO,SECCION,CATEGORIA,SUBCATEGORIA,Week
35,284020,1562372795646,1,700,700,2019-07-05 19:26:00+00:00,7707279031713,CALI,MINIMERCADO,PAPAS ANDINA X 40 GR,40 GR,OTROS FABRICANTES,ANDINA,PGC,PASABOCAS,SNACKS,PAPAS FRITAS,26
244,284020,1562335098564,2,700,1400,2019-07-05 08:58:00+00:00,7707279031713,CALI,MINIMERCADO,PAPAS ANDINA X 40 GR,40 GR,OTROS FABRICANTES,ANDINA,PGC,PASABOCAS,SNACKS,PAPAS FRITAS,26
247,268190,1562196044440,1,800,800,2019-07-03 18:20:00+00:00,7707279031713,CALI,MINIMERCADO,PAPAS ANDINA X 40 GR,40 GR,OTROS FABRICANTES,ANDINA,PGC,PASABOCAS,SNACKS,PAPAS FRITAS,26
278,268190,1562174063372,1,800,800,2019-07-03 12:14:00+00:00,7707279031713,CALI,MINIMERCADO,PAPAS ANDINA X 40 GR,40 GR,OTROS FABRICANTES,ANDINA,PGC,PASABOCAS,SNACKS,PAPAS FRITAS,26
279,268190,1561991158502,1,800,800,2019-07-01 09:25:00+00:00,7707279031713,CALI,MINIMERCADO,PAPAS ANDINA X 40 GR,40 GR,OTROS FABRICANTES,ANDINA,PGC,PASABOCAS,SNACKS,PAPAS FRITAS,26
280,268190,1562199300935,4,800,3200,2019-07-03 19:15:00+00:00,7707279031713,CALI,MINIMERCADO,PAPAS ANDINA X 40 GR,40 GR,OTROS FABRICANTES,ANDINA,PGC,PASABOCAS,SNACKS,PAPAS FRITAS,26
323,263029,1562463011748,1,900,900,2019-07-06 20:30:00+00:00,7707279031713,CALI,MINIMERCADO,PAPAS ANDINA X 40 GR,40 GR,OTROS FABRICANTES,ANDINA,PGC,PASABOCAS,SNACKS,PAPAS FRITAS,26
371,268190,1562467761235,1,800,800,2019-07-06 21:49:00+00:00,7707279031713,CALI,MINIMERCADO,PAPAS ANDINA X 40 GR,40 GR,OTROS FABRICANTES,ANDINA,PGC,PASABOCAS,SNACKS,PAPAS FRITAS,26
378,268190,1562009529248,1,800,800,2019-07-01 14:32:00+00:00,7707279031713,CALI,MINIMERCADO,PAPAS ANDINA X 40 GR,40 GR,OTROS FABRICANTES,ANDINA,PGC,PASABOCAS,SNACKS,PAPAS FRITAS,26
411,268190,1562376137926,1,800,800,2019-07-05 20:22:00+00:00,7707279031713,CALI,MINIMERCADO,PAPAS ANDINA X 40 GR,40 GR,OTROS FABRICANTES,ANDINA,PGC,PASABOCAS,SNACKS,PAPAS FRITAS,26


In [56]:
atransaction.describe()

,cantidad,precioventa,totalventa,Week
count,12.00,12.000000,12.000000,12.0
mean,1.50,791.666667,1183.333333,26.0
std,1.00,51.492865,794.107084,0.0
min,1.00,700.000000,700.000000,26.0
25%,1.00,800.000000,800.000000,26.0
50%,1.00,800.000000,800.000000,26.0
75%,1.25,800.000000,1025.000000,26.0
max,4.00,900.000000,3200.000000,26.0


In [74]:
ventas_d = np.array(list(atransaction['totalventa']))
precioventa =np.array(list(atransaction['precioventa']))
cantidad = np.array(list(atransaction['cantidad']))

### NONE IS GAUSSIAN!!!!

In [60]:
for datt in [ventas_d, precioventa, cantidad]:
    shapiro_t(datt)

Statistics=0.610, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.699, p=0.001
Sample does not look Gaussian (reject H0)
Statistics=0.587, p=0.000
Sample does not look Gaussian (reject H0)


### Same result given the rule you should be eliminating 1/12 of the sample of records; but that accounts for 2/9 of the quantities sold

In [76]:
np.std(ventas_d)*3 + np.median(ventas_d)

3080.8989455914084

### GALLETA SALADAS SALTIN NOEL

In [5]:
atransaction = df[(df["EAN"]=="7702025125173") & (df["Week"] ==27)]

In [6]:
atransaction

,Codigo,Sale_id,cantidad,precioventa,totalventa,Fechatransaccion,EAN,CiudadM,Tipologia_comercial,DESCRIPCION_FINAL,PRESENTACION,FABRICANTE,MARCA,DPTO,SECCION,CATEGORIA,SUBCATEGORIA,Week
64,266619,1562604112631,2,4150,8300,2019-07-08 11:41:00+00:00,7702025125173,BOGOTA,MINIMERCADO,GALLETA SALADAS SALTIN NOEL X 354 GR X 4 UNDS,354 GR,COMPAÑIA DE GALLETAS NOEL SA,SALTIN NOEL,PGC,RANCHO,GALLETAS,GALLETA DE SAL,27
65,263643,1562515492908,1,4100,4100,2019-07-07 11:04:00+00:00,7702025125173,MEDELLIN,MINIMERCADO,GALLETA SALADAS SALTIN NOEL X 354 GR X 4 UNDS,354 GR,COMPAÑIA DE GALLETAS NOEL SA,SALTIN NOEL,PGC,RANCHO,GALLETAS,GALLETA DE SAL,27
122,266619,1562867383724,1,4150,4150,2019-07-11 12:49:00+00:00,7702025125173,BOGOTA,MINIMERCADO,GALLETA SALADAS SALTIN NOEL X 354 GR X 4 UNDS,354 GR,COMPAÑIA DE GALLETAS NOEL SA,SALTIN NOEL,PGC,RANCHO,GALLETAS,GALLETA DE SAL,27
374,263643,1562884419063,1,4100,4100,2019-07-11 17:33:00+00:00,7702025125173,MEDELLIN,MINIMERCADO,GALLETA SALADAS SALTIN NOEL X 354 GR X 4 UNDS,354 GR,COMPAÑIA DE GALLETAS NOEL SA,SALTIN NOEL,PGC,RANCHO,GALLETAS,GALLETA DE SAL,27
375,264428,1562717086219,1,4400,4400,2019-07-09 19:04:00+00:00,7702025125173,MEDELLIN,MINIMERCADO,GALLETA SALADAS SALTIN NOEL X 354 GR X 4 UNDS,354 GR,COMPAÑIA DE GALLETAS NOEL SA,SALTIN NOEL,PGC,RANCHO,GALLETAS,GALLETA DE SAL,27
376,281219,1562847771189,1,4100,4100,2019-07-11 07:22:00+00:00,7702025125173,BUCARAMANGA,TIENDA GRANDE,GALLETA SALADAS SALTIN NOEL X 354 GR X 4 UNDS,354 GR,COMPAÑIA DE GALLETAS NOEL SA,SALTIN NOEL,PGC,RANCHO,GALLETAS,GALLETA DE SAL,27
387,271151,1562944019648,1,4400,4400,2019-07-12 10:07:00+00:00,7702025125173,BOGOTA,MINIMERCADO,GALLETA SALADAS SALTIN NOEL X 354 GR X 4 UNDS,354 GR,COMPAÑIA DE GALLETAS NOEL SA,SALTIN NOEL,PGC,RANCHO,GALLETAS,GALLETA DE SAL,27
514,267863,1563065372678,1,3800,3800,2019-07-13 19:49:00+00:00,7702025125173,BOGOTA,TIENDA GRANDE,GALLETA SALADAS SALTIN NOEL X 354 GR X 4 UNDS,354 GR,COMPAÑIA DE GALLETAS NOEL SA,SALTIN NOEL,PGC,RANCHO,GALLETAS,GALLETA DE SAL,27
517,266619,1562971375763,1,4150,4150,2019-07-12 17:42:00+00:00,7702025125173,BOGOTA,MINIMERCADO,GALLETA SALADAS SALTIN NOEL X 354 GR X 4 UNDS,354 GR,COMPAÑIA DE GALLETAS NOEL SA,SALTIN NOEL,PGC,RANCHO,GALLETAS,GALLETA DE SAL,27
890,284588,1562803724497,1,4200,4200,2019-07-10 19:08:00+00:00,7702025125173,MEDELLIN,MINIMERCADO,GALLETA SALADAS SALTIN NOEL X 354 GR X 4 UNDS,354 GR,COMPAÑIA DE GALLETAS NOEL SA,SALTIN NOEL,PGC,RANCHO,GALLETAS,GALLETA DE SAL,27


In [7]:
ventas_d = np.array(list(atransaction['totalventa']))
precioventa =np.array(list(atransaction['precioventa']))
cantidad = np.array(list(atransaction['cantidad']))

### NONE IS GAUSSIAN!!!!

In [10]:
for datt in [ventas_d, precioventa, cantidad]:
    shapiro_t(datt)

Statistics=0.484, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=0.861, p=0.077
Sample looks Gaussian (fail to reject H0)
Statistics=0.366, p=0.000
Sample does not look Gaussian (reject H0)


### The worst result given the rule you should be eliminating 1/10 of the sample: Terrible. And it is not a mistake

In [12]:
np.std(ventas_d)*3 + np.median(ventas_d)

7910.904146611556

### CEREAL EXPANDIDO FLIPS

In [13]:
atransaction = df[(df["Week"] == 26) & (df["EAN"]=="7702354930974")]

In [14]:
atransaction

,Codigo,Sale_id,cantidad,precioventa,totalventa,Fechatransaccion,EAN,CiudadM,Tipologia_comercial,DESCRIPCION_FINAL,PRESENTACION,FABRICANTE,MARCA,DPTO,SECCION,CATEGORIA,SUBCATEGORIA,Week
138,265093,1562005705282,1,2000,2000,2019-07-01 13:28:00+00:00,7702354930974,PEREIRA,MINIMERCADO,MEZCLA CREMA LA SOPERA PLL.VERD.X90G,90 GR,QUALA SA,LA SOPERA,PGC,RANCHO,SOPAS Y CREMAS,CREMAS,26
140,271856,1562119712892,1,2000,2000,2019-07-02 18:08:00+00:00,7702354930974,BOGOTA,MINIMERCADO,MEZCLA CREMA LA SOPERA PLL.VERD.X90G,90 GR,QUALA SA,LA SOPERA,PGC,RANCHO,SOPAS Y CREMAS,CREMAS,26
175,266296,1562261364759,2,2000,4000,2019-07-04 12:29:00+00:00,7702354930974,BOGOTA,TIENDA GRANDE,MEZCLA CREMA LA SOPERA PLL.VERD.X90G,90 GR,QUALA SA,LA SOPERA,PGC,RANCHO,SOPAS Y CREMAS,CREMAS,26
329,267952,1561996990440,1,2000,2000,2019-07-01 11:03:00+00:00,7702354930974,BOGOTA,TIENDA MEDIANA,MEZCLA CREMA LA SOPERA PLL.VERD.X90G,90 GR,QUALA SA,LA SOPERA,PGC,RANCHO,SOPAS Y CREMAS,CREMAS,26
658,268910,1562171316719,1,2000,2000,2019-07-03 11:28:00+00:00,7702354930974,IBAGUE,TIENDA GRANDE,MEZCLA CREMA LA SOPERA PLL.VERD.X90G,90 GR,QUALA SA,LA SOPERA,PGC,RANCHO,SOPAS Y CREMAS,CREMAS,26
716,30792,1562375736018,1,2000,2000,2019-07-05 20:15:00+00:00,7702354930974,BOGOTA,MINIMERCADO,MEZCLA CREMA LA SOPERA PLL.VERD.X90G,90 GR,QUALA SA,LA SOPERA,PGC,RANCHO,SOPAS Y CREMAS,CREMAS,26
841,30798,1562009948464,1,2000,2000,2019-07-01 14:39:00+00:00,7702354930974,BOGOTA,MINIMERCADO,MEZCLA CREMA LA SOPERA PLL.VERD.X90G,90 GR,QUALA SA,LA SOPERA,PGC,RANCHO,SOPAS Y CREMAS,CREMAS,26
886,269408,1562430414245,1,2000,2000,2019-07-06 11:26:00+00:00,7702354930974,IBAGUE,MINIMERCADO,MEZCLA CREMA LA SOPERA PLL.VERD.X90G,90 GR,QUALA SA,LA SOPERA,PGC,RANCHO,SOPAS Y CREMAS,CREMAS,26


In [15]:
ventas_d = np.array(list(atransaction['totalventa']))
precioventa =np.array(list(atransaction['precioventa']))
cantidad = np.array(list(atransaction['cantidad']))

### NONE IS GAUSSIAN!!!!

In [20]:
for datt in [ventas_d, precioventa, cantidad]:
    shapiro_t(datt)

Statistics=0.418, p=0.000
Sample does not look Gaussian (reject H0)
Statistics=1.000, p=1.000
Sample looks Gaussian (fail to reject H0)
Statistics=0.418, p=0.000
Sample does not look Gaussian (reject H0)


### Given the rule you should be eliminating 1/8 of the sample; far from ideal

In [19]:
np.std(ventas_d)*3 + np.median(ventas_d)

3984.313483298443

### In conclusion you should consider other rules for small sample data. And the question should not be what percentage of data I lost but what percentage of each sample I lost; when samples are small it's really bad